In [ ]:
import joblib
import matplotlib
import numpy as np
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt

from keras.models import load_model
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
%matplotlib inline

alt.data_transformers.disable_max_rows()
alt.renderers.enable('html')

plt.style.use('mpl20')
matplotlib.rcParams['figure.dpi'] = 100
matplotlib.rcParams['figure.figsize'] = 15, 5

import warnings
warnings.filterwarnings('ignore')

In [ ]:
FORECAST_PERIOD = 24 * 3 # 3 days
DECOMPOSED_SHIFT = FORECAST_PERIOD // 2 + (FORECAST_PERIOD // 2) % 24  # round to the closest number of days

FEATURES_LAGS = [24 * 2 + 24 * i for i in range(1, 4)]
RADIATION_LAGS = [24 * 2 + 24 * i for i in range(1, 4)]
RECENT_RADIATION_LAGS = [i for i in range(1, 6)]
MAX_DF_SHIFT = max(FEATURES_LAGS + RADIATION_LAGS)
HOME_DIR = '/content/drive/MyDrive/Colab Notebooks/UCU_ML_2022/UCU_Machine_Learning_Course_Project'

## Init configurations

In [ ]:
# For Google Colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Import the custom model
dir_path = HOME_DIR + '/results/'  # for Google Colab
# dir_path = os.getcwd() + '/'
reconstructed_model = load_model(dir_path + 'custom_model_v3.h5')

# Load MinMaxScaler.
# Since we used it on training, validation and testing datasets, it is better also use the same on the new coming data
delta_scaler = joblib.load(HOME_DIR + '/results/df_scaler_v3.pkl')

# Import trend and seasonality to use them during forecasting
multiplicative_decomposed_trend = pd.read_csv(HOME_DIR + '/results/multiplicative_decomposed_trend_v1.csv',
                                              header = 0, index_col = 0, squeeze = True)
multiplicative_decomposed_seasonal = pd.read_csv(HOME_DIR + '/results/multiplicative_decomposed_seasonal_v1.csv',
                                                 header = 0, index_col = 0, squeeze = True)

# For Google Colab here is path to dataset on Google Drive
hourly_radiation_df = pd.read_csv(HOME_DIR + '/data/dataset1_HourlySolarRadiationProcessed.csv')
# hourly_radiation_df = pd.read_csv(os.path.join("..", "data", "dataset1_HourlySolarRadiationProcessed.csv"))
hourly_radiation_df['Hourly_DateTime'] = pd.to_datetime(hourly_radiation_df['Hourly_DateTime'])
hourly_radiation_df.head()

In [ ]:
delta_features = hourly_radiation_df[-FORECAST_PERIOD:].copy()
forecast_datetime_range = delta_features['Hourly_DateTime']

delta_features = pd.concat([delta_features, delta_features])
delta_features.head()

In [ ]:
def get_extrapolated_trend(trend):
    X = trend.index[:, np.newaxis]
    y = trend.values[:, np.newaxis]

    poly_reg = PolynomialFeatures(degree=3)
    X_poly = poly_reg.fit_transform(X)
    pol_reg = LinearRegression()
    pol_reg.fit(X_poly, y)

    predict_idx = np.array([trend.index[-1] + i for i in range(1, DECOMPOSED_SHIFT + 1 * FORECAST_PERIOD + 1)])
    # predict_idx = np.array([trend.index[-1] + i for i in range(1, 1 * FORECAST_PERIOD + 1)])
    X_actual_and_forecast = np.concatenate((X, predict_idx[:, np.newaxis]))
    polynomial_trend_prediction = pol_reg.predict(poly_reg.fit_transform(X_actual_and_forecast))

    # Visualizing the Polymonial Regression results
    plt.scatter(X, y, color='red')
    plt.plot(X_actual_and_forecast, polynomial_trend_prediction, color='blue')
    plt.ylabel('Production')
    plt.title('Extrapolation of trend')
    plt.show()

    return polynomial_trend_prediction.flatten()


def extrapolate_seasonality(seasonality, num_periods):
    new_seasonality = pd.concat([seasonality, seasonality[:num_periods * 24]])
    new_seasonality.index = range(0, len(new_seasonality))
    plt.plot(new_seasonality)
    plt.title('Extrapolation of seasonality')
    plt.show()
    return new_seasonality

In [ ]:
def create_feature_df_for_delta(df, df_scaler):
    """
    Create dataframe of feature to train models

    :return: normalized feature numpy arrays and scaler instance which was used for normalizations,
        in the future steps it will be used to return to original values from normalized
    """
    feature_lags = [24 * i for i in range(1, 4)]
    radiation_lags = [24 * i for i in range(1, 4)]
    recent_radiation_lags = [i for i in range(1, 6)]
    feature_df = process_delta_data(df, feature_lags, radiation_lags, recent_radiation_lags)
    feature_df.reset_index(drop=True, inplace=True)
    technical_df = pd.Series([1 for _ in range(feature_df.shape[0])])
    full_df = pd.concat([feature_df, technical_df], axis=1)
    scaled_full_df = df_scaler.transform(full_df)

    return scaled_full_df[:, :-1]


def reshape_for_model(model_name, dataset):
    if 'LSTM' in model_name or\
            'RNN' in model_name or\
            'Conv1d' in model_name:

        if 'LSTM' in model_name:
            # reshape input to be 3D [samples, features, timesteps]
            dataset = dataset.reshape((dataset.shape[0], 1, dataset.shape[1]))

        else:
            # reshape input to be 3D [samples, timesteps, features]
            dataset = dataset.reshape((dataset.shape[0], dataset.shape[1], 1))

    return dataset


def reshape_test_set_for_model(model_name, test_X):
    if 'LSTM' in model_name or\
            'RNN' in model_name or\
            'Conv1d' in model_name:

        if 'LSTM' in model_name:
            test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
        else:
            test_X = test_X.reshape((test_X.shape[0], test_X.shape[1]))

    return test_X


def test_model_with_transform(model, test_X, model_name, first_row_idx):
    """

    Use previous predicted value to construct our feature dataframe and use it for next value prediction
    """
    yhat = []

    test_X = reshape_test_set_for_model(model_name, test_X)

    # Take first row for which we start to forecast.
    # Note that first_row_idx param can be also not equal to zero,
    # for example, it can also be the last index in test_X
    test_row = test_X[first_row_idx]
    prod_lags = test_row[-len(RECENT_RADIATION_LAGS):]
    test_row = test_row.reshape((1, len(test_row)))
    test_row = reshape_for_model(model_name, test_row)
    for i in range(len(test_X)):
        yhat_one_pred = model.predict(test_row)
        yhat.append(yhat_one_pred[0])

        test_row = test_X[i]

        prod_lags = np.roll(prod_lags, 1)
        prod_lags[0] = yhat_one_pred
        new_test_row = np.concatenate((test_row[:-len(RECENT_RADIATION_LAGS)], prod_lags), axis=0)
        test_row = new_test_row.reshape((1, len(new_test_row)))
        test_row = reshape_for_model(model_name, test_row)

    yhat = np.array(yhat)
    yhat[yhat<0] = 0
    return yhat

In [ ]:
def process_delta_data(df, features_lags, radiation_lags, recent_radiation_lags):
    """
    Create a feature dataframe, which will be used for training ML and DL models

    :return: tuple, where first element is dataframe of features filtered from NaN rows, which we got as result of lags for different dataframe columns;
                          second element is a timeseries filtered from NaN entries
    """
    features_df = df.copy()
    features_df.drop(['Year', 'Day',
                      'Month',  # we do not have data for the whole year, hence we need to drop 'Month' column
                      'Hour', 'Log_Radiation'], axis=1, inplace=True)
    if 'Hourly_DateTime' in features_df.columns:
        features_df.drop(['Hourly_DateTime'], axis=1, inplace=True)

    # Choose features which has good correlation or good logic causation for solar radiation
    feature_columns = ['Temperature', 'Pressure', 'Humidity', 'ZenithDeviation',
                       'WindDirection(Degrees)', 'Speed']

    print('process_data(): features_df.shape[0] -- ', features_df.shape[0])
    for feature_column_name in feature_columns:
        for lag in features_lags:
            temp = np.concatenate((np.array([np.nan for _ in range(lag)]), features_df[feature_column_name].values[:-lag]))
            features_df[f'{feature_column_name}_lag_{lag}'] = temp

    # Take radiation lags as one of the features
    # Notice that here lags are more than our target forecast period
    for lag in radiation_lags:
        temp = np.concatenate((np.array([np.nan for _ in range(lag)]), features_df['Radiation'].values[:-lag]))
        features_df[f'Radiation_lag_{lag}'] = temp

    # Take last values, which we forecasted, and use them as features also
    for lag in recent_radiation_lags:
        temp = np.concatenate((np.array([np.nan for _ in range(lag)]), features_df['Radiation'].values[:-lag]))
        features_df[f'Radiation_lag_{lag}'] = temp

    features_df.fillna(features_df.mean(), inplace=True)

    # And finally drop rainfalls
    features_df.drop(feature_columns, axis=1, inplace=True)
    features_df.drop(['Radiation'], axis=1, inplace=True)

    print('process_data(): features_df.shape -- ', features_df.shape)
    print('max(features_lags + radiation_lags) -- ', max(features_lags + radiation_lags))

    return features_df[max(features_lags + radiation_lags):]

In [ ]:
def predict_out_of_df(test_model, df_scaler, original_df, test_df, forecast_datetime_range, model_name):
    full_df_X = create_feature_df_for_delta(test_df, df_scaler)

    # reshape input to be 3D [samples, timesteps, features]
    full_df_X = full_df_X.reshape((full_df_X.shape[0], full_df_X.shape[1], 1))

    yhat = test_model_with_transform(test_model, full_df_X, model_name, -1)
    full_df_X = full_df_X.reshape((full_df_X.shape[0], full_df_X.shape[1]))

    # invert scaling for forecast
    inv_yhat = np.concatenate((full_df_X, yhat), axis=1)
    inv_yhat = df_scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:,-1]

    # TODO
    predicted_trend = extrapolate_seasonality(multiplicative_decomposed_seasonal, 3)[-FORECAST_PERIOD:]
    predicted_seasonality = get_extrapolated_trend(multiplicative_decomposed_trend)[-FORECAST_PERIOD:]

    print('inv_yhat.shape[0] -- ', inv_yhat.shape[0])
    print('predicted_trend.shape[0] -- ', predicted_trend.shape[0])
    print('predicted_seasonality.shape[0] -- ', predicted_seasonality.shape[0])
    plt.plot(inv_yhat)
    plt.show()

    model_prediction_initial_series = inv_yhat * predicted_trend * predicted_seasonality

    # start_idx = len(original_df[:-DECOMPOSED_SHIFT])
    start_idx = len(original_df)
    end_idx = start_idx + FORECAST_PERIOD
    print('start_idx -- ', start_idx)
    print('FORECAST_PERIOD -- ', FORECAST_PERIOD)
    print('end_idx -- ', end_idx)
    SHIFT = 0
    # x_indices = range(start_idx, end_idx)
    print("model_prediction_initial_series -- ", model_prediction_initial_series.values)

    # # add + 1 to make consistent plot
    # plt.plot(hourly_radiation_reset_df.Radiation[:-SHIFT], color='blue', label="initial time series")
    # plt.plot(x_indices, model_prediction_initial_series, color='red', label="forecast")
    # plt.legend(loc='upper left')
    # plt.title(f'{model_name} {best_model_params} Prediction on Test Dataset')
    # plt.grid(alpha=0.3)
    # plt.show()

    # add + 1 to make consistent plot
    before_lag = 14 * 24
    # plt.plot(original_df['Hourly_DateTime'][start_idx + SHIFT + 1 - before_lag: start_idx + SHIFT + 1],
    #           original_df['Radiation'][start_idx + SHIFT + 1 - before_lag: start_idx + SHIFT + 1], color='blue', label="last days original radiation")
    # plt.plot(original_df['Hourly_DateTime'][start_idx + SHIFT: end_idx + SHIFT],
    #          model_prediction_initial_series, color='red', label="forecast")

    plt.plot(original_df['Hourly_DateTime'][start_idx + SHIFT + 1 - before_lag: start_idx + SHIFT + 1],
             original_df['Radiation'][start_idx + SHIFT + 1 - before_lag: start_idx + SHIFT + 1], color='blue', label="last days original radiation")
    plt.plot(forecast_datetime_range, model_prediction_initial_series, color='red', label="forecast")
    plt.legend(loc='upper left')
    plt.title(f'Forecast on 3 days ahead')
    plt.grid(alpha=0.3)
    plt.show()

In [ ]:
# Before it run process_data, get_train_validation_test_splits_for_final_test, create_feature_df_for_stationary
# + multiplicative_decomposed_seasonal + multiplicative_decomposed_trend
predict_out_of_df(reconstructed_model, delta_scaler, hourly_radiation_df, delta_features, forecast_datetime_range, model_name='Conv1d')